# Importing the documents

In [3]:
# importing required modules
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import LineTokenizer
import glob
import re
import os
import pandas as pd
import docx2txt
import textract
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import string
from docx.api import Document
#from tabula import read_pdf
#from tabulate import tabulate
import numpy as np


#pandas max columns and rows
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

In [4]:
# Find the current working directory
OCD_df=  pd.read_csv("C:/Users/me1awq/PhD/docsim/datasets/rssb/OCD rulebook.csv", header=None)
OCD_df.columns = ["Source_rules_document"]
OCD_df

,Source_rules_document
0,2.1 Separation of trains
1,Principle: The method of signalling must maintain a space interval between trains that is safe.
2,2.1.1 Context
3,"2.1.1.1 The aim of this principle is to reduce the risk of collision between trains, by establishing a spatial envelope, or interval, around each train. This is normally achieved and maintained by train control and signalling systems, based on one train in one section of line at a time; it applies to train movements on single, double or multiple-track lines."
4,"2.1.1.2 Train control and signalling systems are designed to make the space interval effective against collision with train movements from converging lines, as well as following and opposing movements of trains on the same line."
5,"2.1.1.3 The principle encompasses situations in which, for operational reasons, the space interval between two trains is reduced or eliminated, either as part of regular planned operation (permissive working) or when it is necessary to use a train to assist another train which has failed. It does not include movements solely for the purpose of coupling or uncoupling."
6,2.1.1.4 Operating procedures using written or spoken instructions are applied when the signalling system cannot be used to preserve the effectiveness of the space interval:
7,a) for movements in the opposite direction to that for which the line is signalled
8,"b) when the functionality of the system is partially or totally degraded, due to failure or planned disconnection."
9,2.1.1.5 Authority for train movements under normal and degraded conditions is discussed in section 2.2 of this operational concept document.


In [5]:
# Find the current working directory
EA_df=  pd.read_csv("C:/Users/me1awq/PhD/docsim/datasets/rssb/EA rulebook.csv", header=None, encoding='cp1252')
EA_df.columns = ["Target_rules_document"]
EA_df

,Target_rules_document
0,Driver
1,Driver
2,Driver
3,IM
4,IM
5,IM
6,IM
7,Maintenance
8,Maintenance
9,Maintenance


# Clean the data

In [6]:
# Searching for rule that start with digits or decimal number.
digits_OCD_df = OCD_df[OCD_df.Source_rules_document.str[0].str.isdigit()]

# digits_OCD_df["Source_rules_document_lower"] = digits_OCD_df["Source_rules_document"].str.lower()
digits_OCD_df['Source_rules_document']=digits_OCD_df['Source_rules_document'].str.lower()
digits_OCD_df

C:\Users\me1awq\AppData\Local\Temp\ipykernel_74964\3168557303.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  digits_OCD_df['Source_rules_document']=digits_OCD_df['Source_rules_document'].str.lower()


,Source_rules_document
0,2.1 separation of trains
2,2.1.1 context
3,"2.1.1.1 the aim of this principle is to reduce the risk of collision between trains, by establishing a spatial envelope, or interval, around each train. this is normally achieved and maintained by train control and signalling systems, based on one train in one section of line at a time; it applies to train movements on single, double or multiple-track lines."
4,"2.1.1.2 train control and signalling systems are designed to make the space interval effective against collision with train movements from converging lines, as well as following and opposing movements of trains on the same line."
5,"2.1.1.3 the principle encompasses situations in which, for operational reasons, the space interval between two trains is reduced or eliminated, either as part of regular planned operation (permissive working) or when it is necessary to use a train to assist another train which has failed. it does not include movements solely for the purpose of coupling or uncoupling."
6,2.1.1.4 operating procedures using written or spoken instructions are applied when the signalling system cannot be used to preserve the effectiveness of the space interval:
9,2.1.1.5 authority for train movements under normal and degraded conditions is discussed in section 2.2 of this operational concept document.
10,2.1.2 controls - operation under normal conditions
11,"2.1.2.1 train control and signalling systems must be supported by operating rules, as well as procedures specific to each system, to enable operators to maintain a safe space interval between trains within their area of control when the system is operating under normal conditions."
12,"2.1.2.2 permissive working must be limited to locations where it is authorised by the infrastructure manager. in the case of trains conveying passengers, permissive working must be limited to stations, for the purpose of joining trains or platform sharing."


In [7]:
# Searching for rule that start with digits or decimal number.
digits_EA_df = EA_df[EA_df.Target_rules_document.str[0].str.isdigit()]

# digits_OCD_df["Source_rules_document_lower"] = digits_OCD_df["Source_rules_document"].str.lower()
digits_EA_df['Target_rules_document']=digits_EA_df['Target_rules_document'].str.lower()
digits_EA_df

C:\Users\me1awq\AppData\Local\Temp\ipykernel_74964\1910080252.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  digits_EA_df['Target_rules_document']=digits_EA_df['Target_rules_document'].str.lower()


,Target_rules_document
21,2.1.1 - context
22,2.1.1.1 - establish spatial envelope around each train in one section of line at a time
23,2.1.1.2 - space interval effective against collision
24,"2.1.1.3 - space interval reduced or eliminated for permissive working or train failure, but not coupling or uncoupling"
25,2.1.1.4 - written operating procedures or spoken instructions when the signalling system cannot be used to preserve the effectiveness of the space interval
26,2.1.1.5 - authority of train movements under normal or degraded conditions
27,2.1.2 - control operation under normal conditions
28,2.1.2.1 - im authorises permissive working
29,2.1.2.2 control permissive working location
30,2.1.2.3 reduce likelihood of collisions.due to excessive speed or insufficient space for second train.incorrect train driver response to signal or in-cab indication.incorrect response to signal or spoken message for another train


## Stopwords removal

In [8]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords.words('english')
custom_stopwords = ["â€“","â€™","a)","b)","c)","d)","e)",":","(",")","â€˜","-"]
# punctation=[":","(",")"]
stop.extend(custom_stopwords)
# stop.extend(punctation)

In [9]:
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
digits_OCD_df['Source_rules_document']= digits_OCD_df['Source_rules_document'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
digits_OCD_df.columns = ["Clean_source_rules"]
digits_OCD_df

C:\Users\me1awq\AppData\Local\Temp\ipykernel_74964\3962706436.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  digits_OCD_df['Source_rules_document']= digits_OCD_df['Source_rules_document'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


,Clean_source_rules
0,2.1 separation trains
2,2.1.1 context
3,"2.1.1.1 aim principle reduce risk collision trains, establishing spatial envelope, interval, around train. normally achieved maintained train control signalling systems, based one train one section line time; applies train movements single, double multiple-track lines."
4,"2.1.1.2 train control signalling systems designed make space interval effective collision train movements converging lines, well following opposing movements trains line."
5,"2.1.1.3 principle encompasses situations which, operational reasons, space interval two trains reduced eliminated, either part regular planned operation (permissive working) necessary use train assist another train failed. include movements solely purpose coupling uncoupling."
6,2.1.1.4 operating procedures using written spoken instructions applied signalling system cannot used preserve effectiveness space interval:
9,2.1.1.5 authority train movements normal degraded conditions discussed section 2.2 operational concept document.
10,2.1.2 controls operation normal conditions
11,"2.1.2.1 train control signalling systems must supported operating rules, well procedures specific system, enable operators maintain safe space interval trains within area control system operating normal conditions."
12,"2.1.2.2 permissive working must limited locations authorised infrastructure manager. case trains conveying passengers, permissive working must limited stations, purpose joining trains platform sharing."


In [10]:
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
digits_EA_df['Target_rules_document']= digits_EA_df['Target_rules_document'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
digits_EA_df.columns = ["Clean_target_rules"]
digits_EA_df

C:\Users\me1awq\AppData\Local\Temp\ipykernel_74964\3669078166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  digits_EA_df['Target_rules_document']= digits_EA_df['Target_rules_document'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


,Clean_target_rules
21,2.1.1 context
22,2.1.1.1 establish spatial envelope around train one section line time
23,2.1.1.2 space interval effective collision
24,"2.1.1.3 space interval reduced eliminated permissive working train failure, coupling uncoupling"
25,2.1.1.4 written operating procedures spoken instructions signalling system cannot used preserve effectiveness space interval
26,2.1.1.5 authority train movements normal degraded conditions
27,2.1.2 control operation normal conditions
28,2.1.2.1 im authorises permissive working
29,2.1.2.2 control permissive working location
30,2.1.2.3 reduce likelihood collisions.due excessive speed insufficient space second train.incorrect train driver response signal in-cab indication.incorrect response signal spoken message another train


# Converting the dataframe into a list for both documents

In [11]:
# column to list using Series.tolist()
source_rules = digits_OCD_df['Clean_source_rules'].tolist()
source_rules

['2.1 separation trains',
 '2.1.1 context',
 '2.1.1.1 aim principle reduce risk collision trains, establishing spatial envelope, interval, around train. normally achieved maintained train control signalling systems, based one train one section line time; applies train movements single, double multiple-track lines.',
 '2.1.1.2 train control signalling systems designed make space interval effective collision train movements converging lines, well following opposing movements trains line.',
 '2.1.1.3 principle encompasses situations which, operational reasons, space interval two trains reduced eliminated, either part regular planned operation (permissive working) necessary use train assist another train failed. include movements solely purpose coupling uncoupling.',
 '2.1.1.4 operating procedures using written spoken instructions applied signalling system cannot used preserve effectiveness space interval:',
 '2.1.1.5 authority train movements normal degraded conditions discussed section 2

In [12]:
# column to list using Series.tolist()
target_rules = digits_EA_df['Clean_target_rules'].tolist()
target_rules

['2.1.1 context',
 '2.1.1.1 establish spatial envelope around train one section line time',
 '2.1.1.2 space interval effective collision',
 '2.1.1.3 space interval reduced eliminated permissive working train failure, coupling uncoupling',
 '2.1.1.4 written operating procedures spoken instructions signalling system cannot used preserve effectiveness space interval',
 '2.1.1.5 authority train movements normal degraded conditions',
 '2.1.2 control operation normal conditions',
 '2.1.2.1 im authorises permissive working',
 '2.1.2.2 control permissive working location',
 '2.1.2.3 reduce likelihood collisions.due excessive speed insufficient space second train.incorrect train driver response signal in-cab indication.incorrect response signal spoken message another train',
 '2.1.2.4 assist failed train.prompt accurate failure location reporting (inc. safeguards).clear understanding/protocol signaller train driver regarding direction assistance come',
 '2.1.2.5 control movements opposite direc

# Document mapping: Method 1

In [19]:
#Create an embedding for all the sentences in the documents
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
model = SentenceTransformer('ddobokki/electra-small-nli-sts')
# model = SentenceTransformer("sentence-transformers/stsb-distilbert-base")
# sentence_embeddings = model.encode(sent_tokens)


In [20]:
# #printing the embedding size and shape
# sentence_embeddings1 = model.encode(source_rules)
# print(source_rules[0])
# print(sentence_embeddings1.shape)
# print(sentence_embeddings1[0])


In [21]:
#finding the cosine similarity
def cosine_sim(embeddings1,embeddings2):
    """Find the cosine similarity distance between two vectors."""
    csim=( np.dot(embeddings1,embeddings2) )/ (np.linalg.norm(embeddings1)*np.linalg.norm(embeddings2))
    if np.isnan(np.sum(csim)):
        return 0
    return csim

In [22]:
results=[]

#set the threshold value to get the similairity result accordingly
threshold=0.7

#embedding all the documents and find the similarity between search text and all the tokenize sentences
for target_rule in target_rules:
    sentence_embeddings1 = model.encode(target_rule)

    for source_rule in source_rules:
        sentence_embeddings2 = model.encode(source_rule)
        sim_score1 = cosine_sim(sentence_embeddings1, sentence_embeddings2)

        if sim_score1 > threshold:
                results.append({
                                'Target Rule': target_rule,
                                'Source Rule': source_rule,
                                'Score': sim_score1,              
                })

        results.sort(key=lambda k: k['Score'], reverse=True)
        #results = [set(d.values()) for d in results]
print(results)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Document mapping: Method 2

In [25]:
## BERT BASE

In [13]:
from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('all-MiniLM-L6-v2')

source_rules_embeddings = embedder.encode(source_rules, convert_to_tensor=True)


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(source_rules))
for rule in target_rules:
    target_rule_embedding = embedder.encode(rule, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(target_rule_embedding, source_rules_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    print("\n\n======================\n\n")
    print('\033[1m' + 'Source Rule:'+'\033[0m', rule)
    print('\033[1m' + '\nTop 5 most similar results:' + '\033[0m')


    for score, idx in zip(top_results[0], top_results[1]):
        print(source_rules[idx], "(Score: {:.4f})".format(score))





Source Rule: 2.1.1 context

Top 5 most similar results:
2.1.1 context (Score: 1.0000)
2.2.1 context (Score: 0.9746)
2.3.1 context (Score: 0.9213)
2.4.1 context (Score: 0.8960)
2.5.1 context (Score: 0.8890)




Source Rule: 2.1.1.1 establish spatial envelope around train one section line time

Top 5 most similar results:
2.1.1.2 train control signalling systems designed make space interval effective collision train movements converging lines, well following opposing movements trains line. (Score: 0.5676)
2.1.2.7 interface engineering works moving trains covered section 2.3 operational concept document. (Score: 0.5352)
2.1.2.1 train control signalling systems must supported operating rules, well procedures specific system, enable operators maintain safe space interval trains within area control system operating normal conditions. (Score: 0.5135)
2.1.1.1 aim principle reduce risk collision trains, establishing spatial envelope, interval, around train. normally achieved maintained trai





Source Rule: 2.2.1.1 safety compromised train proceeds without movement autority

Top 5 most similar results:
2.2.1.1 safety benefits system maintaining space intervals trains (see section 2.1 operational concept document) compromised train proceeds without authority movement. (Score: 0.6828)
2.5 conditions safe movement trains (Score: 0.6777)
2.7 hazards safe operation unsafe trains (Score: 0.6451)
2.2.1.4 event train proceeding without authority, signaller may options action try mitigate risk collision derailment. addition, lines trains equipped systems designed intervene automatically, stop reduce speed train, end point within safe distance thereafter (safety overlap) event train driver failing respond correctly imminent end movement authority. systems remove responsibility train driver control train compliance limits movement authority, permissible line speeds, route train-related constraints. (Score: 0.6216)
2.7.2.2 controls applied stop trains approaching location line may ob





Source Rule: 2.3.1.2 level crossing obstruction

Top 5 most similar results:
2.3.1.2 obstruction line level crossing occur when: (Score: 0.8864)
2.3 obstruction line: level crossings infrastructure work (Score: 0.7858)
2.3.2 controls – level crossings (Score: 0.7676)
2.3.2.3 operating rules required, mitigate risks obstructions line normal operation, for: level crossings control signaller crossing keeper automatic user-operated level crossings users required contact supervising signaller crossing large, low slow-moving vehicles animals automatic level crossings requiring local monitoring train drivers. (Score: 0.6683)
2.8.11 controls – members public (level crossings) (Score: 0.6280)




Source Rule: 2.3.1.3 measures reduce collisions level crossings

Top 5 most similar results:
2.3.2 controls – level crossings (Score: 0.6820)
2.3 obstruction line: level crossings infrastructure work (Score: 0.6109)
2.3.2.3 operating rules required, mitigate risks obstructions line normal operation





Source Rule: 2.4.1.2 operational controls.prompt accurate problem reporting.action preventing trains approaching affected location.safe conditions re-establshed train movements

Top 5 most similar results:
2.7.2.2 controls applied stop trains approaching location line may obstructed unsafe referenced section 2.4 operational concept document. (Score: 0.7454)
2.4.2.4 operating rules must specify temporary controls applied train movements when: visual examination site revealed hazard, following report infrastructure defect hazard assessed presenting sufficiently low risk safety trains permit movements resume, work required eliminate hazard hazard found malfunction signalling equipment (requirements authorising train movements signalling system cannot used defects described section 2.2 operational concept document) line affected floodwater snow exceptional rail head conditions. (Score: 0.7282)
2.7.2.3 controls applied protect train crew staff trains adjacent lines, examining working ou





Source Rule: 2.5.1.5 control framework dangerous transport goods

Top 5 most similar results:
2.5.1.5 framework controls transport dangerous goods defined international uk regulations. railway group standards operating rules apply national international regulatory requirements rail operations gb mainline network. (Score: 0.7187)
2.5.3.5 mitigate risks event incident involving dangerous goods, operating rules responsible freight train preparation operation, traffic control, must define requirements for: separating dangerous goods traffic reporting managing dangerous goods incidents. (Score: 0.6760)
2.7.2.7 controls applicable safe loading freight trains identification secure closure dangerous goods consignments referenced section 2.5 operational concept document. (Score: 0.6686)
2.7.2.4 general controls managing dangerous goods incidents referenced section 2.5 operational concept document applicable; controls protect train crew adverse effects health resulting release dangerous good





Source Rule: 2.5.3.5 mitigate risk incident dangerous goods

Top 5 most similar results:
2.5.3.5 mitigate risks event incident involving dangerous goods, operating rules responsible freight train preparation operation, traffic control, must define requirements for: separating dangerous goods traffic reporting managing dangerous goods incidents. (Score: 0.7674)
2.7.1.9 dangerous goods incident may result fire, collision derailment, well defect vehicle carrying substance. chain reporting normally involve specialist technical advisor emergency services: specialists may needed resolve incident confirm site safe. then, trains adjacent lines may stopped and, cases, people properties near railway evacuated. (Score: 0.5846)
2.7.2.4 general controls managing dangerous goods incidents referenced section 2.5 operational concept document applicable; controls protect train crew adverse effects health resulting release dangerous goods must also defined operating rules operating instructions conv





Source Rule: 2.6.2 controls

Top 5 most similar results:
2.6.2 controls (Score: 1.0000)
2.2.2 controls (Score: 0.8879)
2.7.2 controls (Score: 0.8529)
2.4.2 controls (Score: 0.8379)
2.9.2 controls (Score: 0.8051)




Source Rule: 2.6.2.1 vehicle information including braking capacity provided (2.5)

Top 5 most similar results:
2.5.2.4 sufficiency operation braking system must assured winter weather conditions company instructions specific train. (Score: 0.5981)
2.6.2.1 controls outlined section 2.5 operational concept document applicable principle, respect of: sufficiency braking capacity information required train preparers operating staff vehicle train. (Score: 0.5906)
2.7.2.6 requirements checking continuity automatic brake referenced section 2.5 operational concept document. (Score: 0.5835)
2.5.2.6 information must made available people responsible train preparation permitted speed vehicle train, train drivers must informed permitted speed complete train. (Score: 0.4860)
2.5.2.1





Source Rule: 2.7.2.11 loads displaced and/or insecure (2.5 movement vehicles)

Top 5 most similar results:
2.7.2.11 procedures managing incidents loads become displaced insecure must prescribed operating rules. conditions applicable movement vehicles whose load may obstruct adjacent line exceed infrastructure gauge referenced section 2.5 operational concept document. (Score: 0.6469)
2.7.1.10 displaced insecure load freight vehicle potential collision another train lineside structures derailment. trains adjacent line must stopped. usually preferable move vehicle load special conditions location clear running lines, replaced secured correctly, transhipped, rather increase delay trains. (Score: 0.6247)
2.6.1.9 cases, risks arising incompatibility infrastructure mitigated applying special conditions travel train movement (exceptional loads procedures), as: (Score: 0.5546)
2.5 conditions safe movement trains (Score: 0.5104)
2.6.1.10 infrastructure manager responsible defining conditions





Source Rule: 2.8.1.5 move rail-borne plan, on-track machines trains maintenance area

Top 5 most similar results:
2.8.1.5 rail-borne plant, on-track machines trains essential infrastructure work may moved within areas designated protected safety staff. controls necessary, form warnings restrictions speed movements. (Score: 0.7416)
2.3.1.7 movement trains associated infrastructure work, including on-track machines plant, within blocked section line, subject similar derailment collision hazards train movements past site work. (Score: 0.6756)
2.8.5 controls – railway workforce (examination repair train rail vehicle) (Score: 0.6695)
2.5.3.2 operating rules operating instructions on-track machines on-track plant must prescribe safety checks carried prior movement. on-track plant permitted moved rail mode outside possessions. (Score: 0.6656)
2.8.2 controls – railway workforce (all activities near line) (Score: 0.6592)




Source Rule: 2.8.1.6 work open line adhering procedures (restricte





Source Rule: 2.9.1 context

Top 5 most similar results:
2.9.1 context (Score: 1.0000)
2.8.1 context (Score: 0.8401)
2.2.1 context (Score: 0.8258)
2.1.1 context (Score: 0.8252)
2.3.1 context (Score: 0.8219)




Source Rule: 2.9.1.1 electric traction' overhead lines conductor rails hazard

Top 5 most similar results:
2.9.1.1 principle addresses hazards railway workforce electric traction current supply systems using overhead lines conductor rails. concerned technical details systems. (Score: 0.7474)
2.9.1.3 electrified lines, particular hazards electrical equipment require additional controls safeguard railway workers; additional controls common electrification systems. (Score: 0.5510)
2.8.1.10 hazards moving defective rail vehicles covered section 2.7 operational concept document. (Score: 0.5214)
2.3.1.6 work maintain, repair replace railway infrastructure may obstruct make line unsafe trains pass it. hazards may result nature work (for example, removal rails) equipment materials re

In [14]:
## Distilroberta BASE

In [15]:
from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('msmarco-distilroberta-base-v2')
source_rules_embeddings = embedder.encode(source_rules, convert_to_tensor=True)


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(source_rules))
for rule in target_rules:
    target_rule_embedding = embedder.encode(rule, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(target_rule_embedding, source_rules_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    print("\n\n======================\n\n")
    print('\033[1m' + 'Source Rule:'+'\033[0m', rule)
    print('\033[1m' + '\nTop 5 most similar results:' + '\033[0m')


    for score, idx in zip(top_results[0], top_results[1]):
        print(source_rules[idx], "(Score: {:.4f})".format(score))





Source Rule: 2.1.1 context

Top 5 most similar results:
2.1.1 context (Score: 1.0000)
2.2.1 context (Score: 0.9879)
2.6.1 context (Score: 0.9817)
2.3.1 context (Score: 0.9777)
2.7.1 context (Score: 0.9772)




Source Rule: 2.1.1.1 establish spatial envelope around train one section line time

Top 5 most similar results:
2.1.1.2 train control signalling systems designed make space interval effective collision train movements converging lines, well following opposing movements trains line. (Score: 0.6244)
2.1.2.1 train control signalling systems must supported operating rules, well procedures specific system, enable operators maintain safe space interval trains within area control system operating normal conditions. (Score: 0.6086)
2.1.1.1 aim principle reduce risk collision trains, establishing spatial envelope, interval, around train. normally achieved maintained train control signalling systems, based one train one section line time; applies train movements single, double multiple





Source Rule: 2.1.3.1 train control signalling partially totally degraded -> operating procedures

Top 5 most similar results:
2.1.3.1 functionality train control signalling system partially totally degraded, operating rules must define operating procedures necessary ensure maintenance safe space interval trains. (Score: 0.7545)
2.5.2 controls – trains (Score: 0.7115)
2.7.1.3 controls intended prevent trains starting journey leaving maintenance location defects detected routine maintenance, checks made entering service. (Score: 0.6901)
2.1.1.2 train control signalling systems designed make space interval effective collision train movements converging lines, well following opposing movements trains line. (Score: 0.6795)
2.1.2.1 train control signalling systems must supported operating rules, well procedures specific system, enable operators maintain safe space interval trains within area control system operating normal conditions. (Score: 0.6594)




Source Rule: 2.1.3.2 cover degrad





Source Rule: 2.2.1.7 signalling system cannot used authorise movement third party maybe used pass movement instructions confirm successfully completed

Top 5 most similar results:
2.2.1.5 signalling system cannot used authorise movement define end point, spoken written instructions used instead, example. (Score: 0.8492)
2.2.2.1 signalling systems must supported operating rules well procedures specific system, define movement authority given: signalling system functioning normally signalling system cannot used authorise movement because: (i) movement made opposite direction line signalled (ii) movement made section line already occupied train, permissive working available (iii) movement enter, leave take place wholly within possession line granted allow engineering work take place (iv) movement permit examination line (v) functionality system partially totally degraded. shunting movement made. (Score: 0.7813)
2.2.1.7 signalling system cannot used authorise movement, third party, pil





Source Rule: 2.3.1.10 authorising train movements (into withing) moved person charge blocked section. leaving blocked section moved back signaller

Top 5 most similar results:
2.3.1.10 responsibility authorising train movements within blocked line transferred signaller person appointed take charge blocked section, also responsible confirming signaller blockage set planned permitting work start. trains cannot leave blocked section without signaller’s permission. (Score: 0.7317)
2.2.2.2 operating rules, together procedures specific system, must provided use signalling system cannot used authorise movement indicate end point. rules must ensure involved authorising carrying train movements understand clearly: responsible authorising movements means authority given train driver end point movement special conditions applicable it, restrictions speed need successful completion movement reported signaller. (Score: 0.6270)
2.2.1.4 event train proceeding without authority, signaller may opti





Source Rule: 2.3.2 controls level crossings

Top 5 most similar results:
2.3.2 controls – level crossings (Score: 0.9801)
2.8.11 controls – members public (level crossings) (Score: 0.7207)
2.5.2 controls – trains (Score: 0.7084)
2.8.8.1 operators manual level crossing gates must provided instructions specific crossing concerned, including type interface level crossing signalling system. (Score: 0.6852)
2.6.2 controls (Score: 0.6789)




Source Rule: 2.3.2.1 control different level crossing types, lineside signs signalling equipment

Top 5 most similar results:
2.3.2.1 operating rules must publish descriptions types level crossing use lineside signs signalling equipment affecting train movements towards crossings. (Score: 0.8071)
2.3.2 controls – level crossings (Score: 0.6724)
2.8.8.1 operators manual level crossing gates must provided instructions specific crossing concerned, including type interface level crossing signalling system. (Score: 0.6622)
2.3 obstruction line: level cro

...





Source Rule: 2.7.2 controls

Top 5 most similar results:
2.7.2 controls (Score: 1.0000)
2.6.2 controls (Score: 0.9867)
2.9.2 controls (Score: 0.9865)
2.4.2 controls (Score: 0.9827)
2.2.2 controls (Score: 0.9768)




Source Rule: 2.7.2.1 promptly report train defect, fires dangerous goods incident displaced loads

Top 5 most similar results:
2.7.1.9 dangerous goods incident may result fire, collision derailment, well defect vehicle carrying substance. chain reporting normally involve specialist technical advisor emergency services: specialists may needed resolve incident confirm site safe. then, trains adjacent lines may stopped and, cases, people properties near railway evacuated. (Score: 0.6164)
2.1.2.4 operating rules assisting failed train (where permissive working available), must include: requirement prompt accurate reporting failure location requirement establishing clear understanding signaller train driver failed train direction assistance come safeguards possible errors lo





Source Rule: 2.7.2.7 freight trains safe loading inc dangerous good identification secure closure (2.5)

Top 5 most similar results:
2.7.2.7 controls applicable safe loading freight trains identification secure closure dangerous goods consignments referenced section 2.5 operational concept document. (Score: 0.8417)
2.7 hazards safe operation unsafe trains (Score: 0.7162)
2.5.3.6 freight train drivers must provided following information starting journey. formation train, including brake force, weight length. lowest maximum permitted speed vehicle train. route availability vehicles train. details dangerous goods conveyed. special conditions applicable movement load vehicle conveyed. f) vehicles train defects requiring reduction speed train. g) vehicles train without operative automatic brakes. information must updated formation changed status vehicles changed (for example, loaded empty). (Score: 0.6722)
2.7.1.10 displaced insecure load freight vehicle potential collision another trai





Source Rule: 2.8.1.14 hazard potentially effecting train safety (2.3)

Top 5 most similar results:
2.8.1.14 potential hazards safety trains actions road users level crossings covered section 2.3 operational concept document. (Score: 0.7991)
2.7 hazards safe operation unsafe trains (Score: 0.7724)
2.4.1.8 protection trains identified hazard remains paramount but, within boundaries requirement, desirable train movements resumed soon possible mitigate adverse effects railway operations congestion, delay, prolonged operation degraded conditions, may increase likelihood human error. (Score: 0.7488)
2.4.1.4 establishing precise nature hazard potential impact safety train movements may require information, obtained technical operational assessment. circumstances, train driver may asked check report section line driving (an examination line). (Score: 0.6922)
2.8.1.10 hazards moving defective rail vehicles covered section 2.7 operational concept document. (Score: 0.6731)




Source Rule: 2.





Source Rule: 2.8.2 control near line

Top 5 most similar results:
2.3.2 controls – level crossings (Score: 0.6652)
2.2.2 controls (Score: 0.6438)
2.4.2 controls (Score: 0.6363)
2.6.2 controls (Score: 0.6304)
2.7.2 controls (Score: 0.6224)




Source Rule: 2.8.2.1 warning protecting people procedures

Top 5 most similar results:
2.8.9.1 operating rules must enable platform staff protected signalling system controlling affected line, objects retrieved. (Score: 0.5508)
2.7.2.10 actions taken response report dangerous goods incident must prescribed operating rules, supported instructions specific goods involved, provided consignor transport undertaking. (Score: 0.5164)
2.8.3.2 people responsible implementing managing procedures protect warn people carrying infrastructure work must provided local information enable apply effective protection warning arrangements. (Score: 0.5150)
2.2.1.6 shunting movements, authority extent may conveyed entirely spoken instructions hand signals, may also





Source Rule: 2.8.8 control manual operation level crossing

Top 5 most similar results:
2.3.2 controls – level crossings (Score: 0.8216)
2.8.8.1 operators manual level crossing gates must provided instructions specific crossing concerned, including type interface level crossing signalling system. (Score: 0.7480)
2.8.8 controls – railway workforce (manual operation level crossing gates) (Score: 0.6561)
2.3.2.3 operating rules required, mitigate risks obstructions line normal operation, for: level crossings control signaller crossing keeper automatic user-operated level crossings users required contact supervising signaller crossing large, low slow-moving vehicles animals automatic level crossings requiring local monitoring train drivers. (Score: 0.6223)
2.5.2 controls – trains (Score: 0.6205)




Source Rule: 2.8.9 control retrieving objects platform line station

Top 5 most similar results:
2.8.9 controls – railway workforce (retrieving objects platform line station) (Score: 0.7029

In [16]:
## Electra Small

In [17]:
from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('ddobokki/electra-small-nli-sts')

source_rules_embeddings = embedder.encode(source_rules, convert_to_tensor=True)


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(source_rules))
for rule in target_rules:
    target_rule_embedding = embedder.encode(rule, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(target_rule_embedding, source_rules_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    print("\n\n======================\n\n")
    print('\033[1m' + 'Source Rule:'+'\033[0m', rule)
    print('\033[1m' + '\nTop 5 most similar results:' + '\033[0m')


    for score, idx in zip(top_results[0], top_results[1]):
        print(source_rules[idx], "(Score: {:.4f})".format(score))





Source Rule: 2.1.1 context

Top 5 most similar results:
2.1.1 context (Score: 1.0000)
2.2.1 context (Score: 0.9779)
2.3.1 context (Score: 0.9603)
2.4.1 context (Score: 0.9197)
2.5.1 context (Score: 0.8714)




Source Rule: 2.1.1.1 establish spatial envelope around train one section line time

Top 5 most similar results:
2.5.1.1 principle concerns train readiness movement. encompasses: (Score: 0.8913)
2.1.2.1 train control signalling systems must supported operating rules, well procedures specific system, enable operators maintain safe space interval trains within area control system operating normal conditions. (Score: 0.8574)
2.1.1.1 aim principle reduce risk collision trains, establishing spatial envelope, interval, around train. normally achieved maintained train control signalling systems, based one train one section line time; applies train movements single, double multiple-track lines. (Score: 0.8512)
2.5.3.7 equipment mounted on-track machines, engineering plant removed poss





Source Rule: 2.2.1 context

Top 5 most similar results:
2.2.1 context (Score: 1.0000)
2.3.1 context (Score: 0.9794)
2.1.1 context (Score: 0.9779)
2.4.1 context (Score: 0.9446)
2.2.2 controls (Score: 0.9016)




Source Rule: 2.2.1.1 safety compromised train proceeds without movement autority

Top 5 most similar results:
2.2.1.1 safety benefits system maintaining space intervals trains (see section 2.1 operational concept document) compromised train proceeds without authority movement. (Score: 0.9173)
2.2.2.4 operating rules must include protocols clear communication involved procedures authorise train movements, using spoken written instructions. (Score: 0.8936)
2.6.1.1 principle concerned confirming compatibility train infrastructure route planned operate, movement authorised. (Score: 0.8910)
2.5.1.3 safety infrastructure train travel subject sections 2.3 2.4 operational concept document. compatibility rail vehicles infrastructure subject section 2.6 operational concept document. (





Source Rule: 2.3.1.4 level crossing operating rules

Top 5 most similar results:
2.3.2 controls – level crossings (Score: 0.9031)
2.3.1.2 obstruction line level crossing occur when: (Score: 0.8913)
2.3.2.2 route information provided train drivers must include location level crossings. (Score: 0.8768)
2.3.1.4 operating rules used: (Score: 0.8684)
2.5.1.4 significant hazardous events addressed section are: (Score: 0.8675)




Source Rule: 2.3.1.6 maintenance obstruction operating rules

Top 5 most similar results:
2.4.1.7 mitigation risks trains arising work remove hazard restore infrastructure subject section 2.3 operational concept document. (Score: 0.9038)
2.5.1.2 addition, train must authority movement (see section 2.2 operational concept document). (Score: 0.8923)
2.1.1.5 authority train movements normal degraded conditions discussed section 2.2 operational concept document. (Score: 0.8890)
2.7.2.2 controls applied stop trains approaching location line may obstructed unsafe refe





Source Rule: 2.4.2.1 control hazards train movements

Top 5 most similar results:
2.5.2 controls – trains (Score: 0.8866)
2.5.4 controls – passenger trains (Score: 0.8743)
2.5.3 controls – freight trains (Score: 0.8722)
2.1 separation trains (Score: 0.8696)
2.5 conditions safe movement trains (Score: 0.8657)




Source Rule: 2.4.2.2 accurately assess impact safe movement

Top 5 most similar results:
2.5.1.2 addition, train must authority movement (see section 2.2 operational concept document). (Score: 0.8238)
2.2 movement authority train (Score: 0.8229)
2.1.1.5 authority train movements normal degraded conditions discussed section 2.2 operational concept document. (Score: 0.8163)
2.3.1.2 obstruction line level crossing occur when: (Score: 0.8072)
2.3.2.2 route information provided train drivers must include location level crossings. (Score: 0.8065)




Source Rule: 2.4.2.3 use trains examine line harzard

Top 5 most similar results:
2.5.2 controls – trains (Score: 0.8650)
2.3.2 con





Source Rule: 2.5.3 controls freight trains

Top 5 most similar results:
2.5.3 controls – freight trains (Score: 0.9904)
2.5.2 controls – trains (Score: 0.9523)
2.5.4 controls – passenger trains (Score: 0.9314)
2.5 conditions safe movement trains (Score: 0.9104)
2.3.2 controls – level crossings (Score: 0.8625)




Source Rule: 2.5.3.1 load prepare

Top 5 most similar results:
2.5.1 context (Score: 0.8372)
2.5.2 controls – trains (Score: 0.8358)
2.3.2 controls – level crossings (Score: 0.8229)
2.5.3 controls – freight trains (Score: 0.8218)
2.4.1 context (Score: 0.8141)




Source Rule: 2.5.3.2 control on-track machines plant operation safety checks

Top 5 most similar results:
2.5.3.2 operating rules operating instructions on-track machines on-track plant must prescribe safety checks carried prior movement. on-track plant permitted moved rail mode outside possessions. (Score: 0.9034)
2.5.2.4 sufficiency operation braking system must assured winter weather conditions company instruct





Source Rule: 2.6.1.5 express route availability number (vehicle infrastructure)

Top 5 most similar results:
2.6.1.4 route availability (ra) system provides consistent simple method assessing compatibility weight rail vehicles capacity underline bridges (other bridges defined ‘long span’). (Score: 0.8839)
2.3.1.5 principle include pedestrian (footpath) crossings: interface members public trains level crossings referenced section 2.8 operational concept document. (Score: 0.8663)
2.4.1.7 mitigation risks trains arising work remove hazard restore infrastructure subject section 2.3 operational concept document. (Score: 0.8632)
2.3.2.2 route information provided train drivers must include location level crossings. (Score: 0.8591)
2.5.2.6 information must made available people responsible train preparation permitted speed vehicle train, train drivers must informed permitted speed complete train. (Score: 0.8576)




Source Rule: 2.6.1.6 freight wagon ra based payload weight

Top 5 most si





Source Rule: 2.6.2.5 train door operation (platform short)

Top 5 most similar results:
2.4.1.7 mitigation risks trains arising work remove hazard restore infrastructure subject section 2.3 operational concept document. (Score: 0.8920)
2.7.2.6 requirements checking continuity automatic brake referenced section 2.5 operational concept document. (Score: 0.8910)
2.5.1.2 addition, train must authority movement (see section 2.2 operational concept document). (Score: 0.8895)
2.1.2.7 interface engineering works moving trains covered section 2.3 operational concept document. (Score: 0.8672)
2.5.3 controls – freight trains (Score: 0.8664)




Source Rule: 2.7.1 context

Top 5 most similar results:
2.7.1 context (Score: 1.0000)
2.7.2 controls (Score: 0.9400)
2.7.1.2 three primary controls are: (Score: 0.9021)
2.8.1 context (Score: 0.8561)
2.6.1 context (Score: 0.8369)




Source Rule: 2.7.1.1 train hazard presented

Top 5 most similar results:
2.7.1.2 three primary controls are: (Score: 0.90





Source Rule: 2.7.19 dangerous goods incident evacuation

Top 5 most similar results:
2.7.2.4 general controls managing dangerous goods incidents referenced section 2.5 operational concept document applicable; controls protect train crew adverse effects health resulting release dangerous goods must also defined operating rules operating instructions conveying specific types dangerous goods. (Score: 0.7707)
2.7.2.10 actions taken response report dangerous goods incident must prescribed operating rules, supported instructions specific goods involved, provided consignor transport undertaking. (Score: 0.7665)
2.5.1.4 significant hazardous events addressed section are: (Score: 0.7651)
2.2.1.7 signalling system cannot used authorise movement, third party, pilotman handsignaller, may used to: (Score: 0.7638)
2.5.3.5 mitigate risks event incident involving dangerous goods, operating rules responsible freight train preparation operation, traffic control, must define requirements for: separat





Source Rule: 2.7.2.8 move defective train mainline railway

Top 5 most similar results:
2.8.7 controls – railway workforce (train drivers changing cabs) (Score: 0.8835)
2.8.2 controls – railway workforce (all activities near line) (Score: 0.8646)
2.8.3 controls – railway workforce (infrastructure work) (Score: 0.8605)
2.8.5 controls – railway workforce (examination repair train rail vehicle) (Score: 0.8487)
2.8.1.7 section 2.3 operational concept document covers hazards trains arise infrastructure work. (Score: 0.8392)




Source Rule: 2.7.2.9 respond train fires

Top 5 most similar results:
2.8.7 controls – railway workforce (train drivers changing cabs) (Score: 0.8684)
2.8.1.7 section 2.3 operational concept document covers hazards trains arise infrastructure work. (Score: 0.8670)
2.5.2.9 controls applicable rail vehicles defective covered section 2.7 operational concept document. (Score: 0.8668)
2.8.9 controls – railway workforce (retrieving objects platform line station) (Score





Source Rule: 2.8.3.4 people working alone

Top 5 most similar results:
2.8.3.4 operating rules must include procedures applied people working alone. (Score: 0.8740)
2.8.2 controls – railway workforce (all activities near line) (Score: 0.8248)
2.8.3 controls – railway workforce (infrastructure work) (Score: 0.7897)
2.8.1 context (Score: 0.7896)
2.7.2 controls (Score: 0.7741)




Source Rule: 2.8.4 control manual coupling uncoupling shunting movements rail vehicles

Top 5 most similar results:
2.8.4 controls – railway workforce (manual coupling uncoupling rail vehicles controlling shunting movements) (Score: 0.9664)
2.8.5 controls – railway workforce (examination repair train rail vehicle) (Score: 0.9470)
2.8.8 controls – railway workforce (manual operation level crossing gates) (Score: 0.9387)
2.8.2 controls – railway workforce (all activities near line) (Score: 0.9260)
2.8.6 controls – railway workforce (communication train drivers signallers) (Score: 0.9255)




Source Rule: 2.8.5

In [18]:
## MPNET

In [19]:
from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('all-mpnet-base-v2')

source_rules_embeddings = embedder.encode(source_rules, convert_to_tensor=True)


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(source_rules))
for rule in target_rules:
    target_rule_embedding = embedder.encode(rule, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(target_rule_embedding, source_rules_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    print("\n\n======================\n\n")
    print('\033[1m' + 'Source Rule:'+'\033[0m', rule)
    print('\033[1m' + '\nTop 5 most similar results:' + '\033[0m')


    for score, idx in zip(top_results[0], top_results[1]):
        print(source_rules[idx], "(Score: {:.4f})".format(score))





Source Rule: 2.1.1 context

Top 5 most similar results:
2.1.1 context (Score: 1.0000)
2.2.1 context (Score: 0.9685)
2.3.1 context (Score: 0.8889)
2.4.1 context (Score: 0.8835)
2.5.1 context (Score: 0.8695)




Source Rule: 2.1.1.1 establish spatial envelope around train one section line time

Top 5 most similar results:
2.1.1.1 aim principle reduce risk collision trains, establishing spatial envelope, interval, around train. normally achieved maintained train control signalling systems, based one train one section line time; applies train movements single, double multiple-track lines. (Score: 0.5908)
2.2.1.1 safety benefits system maintaining space intervals trains (see section 2.1 operational concept document) compromised train proceeds without authority movement. (Score: 0.4959)
2.1.2.7 interface engineering works moving trains covered section 2.3 operational concept document. (Score: 0.4819)
2.8.1.1 principle addresses interface people moving trains: (Score: 0.4720)
2.1.1.2 trai





Source Rule: 2.1.2.7 control engineering works interface moving trains

Top 5 most similar results:
2.1.2.7 interface engineering works moving trains covered section 2.3 operational concept document. (Score: 0.8357)
2.5.2 controls – trains (Score: 0.8247)
2.5.3 controls – freight trains (Score: 0.8034)
2.5.4 controls – passenger trains (Score: 0.7891)
2.8.1.1 principle addresses interface people moving trains: (Score: 0.7670)




Source Rule: 2.1.3 control operations degraded conditions

Top 5 most similar results:
2.1.3 controls – operations degraded conditions (Score: 0.9501)
2.1.2 controls operation normal conditions (Score: 0.6316)
2.1.3.1 functionality train control signalling system partially totally degraded, operating rules must define operating procedures necessary ensure maintenance safe space interval trains. (Score: 0.5664)
2.1.1.5 authority train movements normal degraded conditions discussed section 2.2 operational concept document. (Score: 0.4895)
2.5.2.9 controls ap





Source Rule: 2.2.2.2 control operating procedures system signalling system cannot used authorise movement

Top 5 most similar results:
2.2.1.7 signalling system cannot used authorise movement, third party, pilotman handsignaller, may used to: (Score: 0.7918)
2.2.1.5 signalling system cannot used authorise movement define end point, spoken written instructions used instead, example. (Score: 0.7671)
2.2.2.1 signalling systems must supported operating rules well procedures specific system, define movement authority given: signalling system functioning normally signalling system cannot used authorise movement because: (i) movement made opposite direction line signalled (ii) movement made section line already occupied train, permissive working available (iii) movement enter, leave take place wholly within possession line granted allow engineering work take place (iv) movement permit examination line (v) functionality system partially totally degraded. shunting movement made. (Score: 0.7





Source Rule: 2.3.1.6 maintenance obstruction operating rules

Top 5 most similar results:
2.3.1.4 operating rules used: (Score: 0.6150)
2.8.3.3 operating rules must prescribe conditions applied movements engineering trains on-track plant machinery area protected safety workforce. (Score: 0.6113)
2.3.3.1 operating rules managing infrastructure work must define: criteria deciding whether work requires blockage line done without blocking line trains responsibilities signallers people charge arrangements blocking line controlling movements trains towards, into, within blockage procedures blocking line prevent trains entering blockage (unless required work) procedures preventing trains outside blockage colliding equipment materials use within blockage procedures control train movements entrance exit blocked section line f) procedures control movements trains on-track plant within blockage, prevent: (i) collisions within blockage (ii) unauthorised movements blockage towards line open tra





Source Rule: 2.4.1.1 controls prevent threat train movements presented obstruction unsafe condition

Top 5 most similar results:
2.4.1.1 starting point principle section line known suspected unsafe trains pass it. controls ensure safety infrastructure train movements overcome circumvented; controls described designed prevent threat train movements presented obstruction unsafe condition realised. (Score: 0.8784)
2.4.1.2 operational controls contain mitigate risks trains unsafe condition obstruction line address three principal elements. (Score: 0.8223)
2.8.9.2 object could present hazard train obstruction within scope section 2.4 document, requiring immediate action prevent approach trains. (Score: 0.7884)
2.7.2.2 controls applied stop trains approaching location line may obstructed unsafe referenced section 2.4 operational concept document. (Score: 0.7853)
2.7.2.3 controls applied protect train crew staff trains adjacent lines, examining working outside defective train, referenced 

...





Source Rule: 2.7.2.6 check brake continuity (2.5)

Top 5 most similar results:
2.7.2.6 requirements checking continuity automatic brake referenced section 2.5 operational concept document. (Score: 0.7645)
2.5.2.3 continuity operation brake system throughout train must tested, either prepared service starts journey, work correct brake defect journey. brake continuity must also proved testing accordance relevant rolling stock requirements formation train changed journey. (Score: 0.6953)
2.5.2.5 requirements testing brake continuity must prescribed operating rules. test procedures appropriate types train vehicle operated must provided train preparation operating staff. (Score: 0.6440)
2.5.2.4 sufficiency operation braking system must assured winter weather conditions company instructions specific train. (Score: 0.4954)
2.5.2.1 trains must permitted start journey unless sufficient braking capacity available complete train. (Score: 0.4512)




Source Rule: 2.7.2.7 freight trains safe lo





Source Rule: 2.8.1.4 physically separate people moving trains. warn people approaching trains (protection warning procedures)

Top 5 most similar results:
2.8.1.4 keep safe distance moving trains people near line necessary physically separate work moving trains (protection procedures) or, separation cannot provided, warn approaching train, sufficient time stop work move position safety (warning procedures). (Score: 0.8974)
2.8 keeping people away moving trains (Score: 0.7974)
2.3.3.3 protection people working near line hazards presented moving trains covered section 2.8 operational concept document. (Score: 0.7655)
2.8.1.6 working line open trains relying warning procedures places constraints type work done (it must compromise infrastructure subsystems – track signalling) equipment used (it must easily quickly portable position form obstruction passing train). need easily accessible positions safety trains approach restricts locations warning procedures used. (Score: 0.7511)
2.8.2.





Source Rule: 2.8.2 control near line

Top 5 most similar results:
2.8.2 controls – railway workforce (all activities near line) (Score: 0.5223)
2.9.1.4 additional controls involve: (Score: 0.4516)
2.7.1.2 three primary controls are: (Score: 0.4467)
2.3.2 controls – level crossings (Score: 0.4357)
2.8.9.1 operating rules must enable platform staff protected signalling system controlling affected line, objects retrieved. (Score: 0.4262)




Source Rule: 2.8.2.1 warning protecting people procedures

Top 5 most similar results:
2.8.3.2 people responsible implementing managing procedures protect warn people carrying infrastructure work must provided local information enable apply effective protection warning arrangements. (Score: 0.6306)
2.8.3.4 operating rules must include procedures applied people working alone. (Score: 0.5600)
2.8.5.1 operating rules must define procedures protecting: train drivers examine outside train adjacent running line technical staff gain access work outside v





Source Rule: 2.9.1.1 electric traction' overhead lines conductor rails hazard

Top 5 most similar results:
2.9.1.1 principle addresses hazards railway workforce electric traction current supply systems using overhead lines conductor rails. concerned technical details systems. (Score: 0.7933)
2.9.1.3 electrified lines, particular hazards electrical equipment require additional controls safeguard railway workers; additional controls common electrification systems. (Score: 0.6488)
2.8.1.14 potential hazards safety trains actions road users level crossings covered section 2.3 operational concept document. (Score: 0.6148)
2.8.2.1 operating rules safety people near line must include: requirements (which may general task-specific application) personal safety individuals work groups definition terms used identify common features railway infrastructure equipment explanation ‘on near line’ explanation signs used warn workforce common hazards safety prohibit access area unless protection proc





Source Rule: 2.3 obstruction line: level crossings infrastructure work

Top 5 most similar results:
2.3 obstruction line: level crossings infrastructure work (Score: 1.0000)
2.3.2.3 operating rules required, mitigate risks obstructions line normal operation, for: level crossings control signaller crossing keeper automatic user-operated level crossings users required contact supervising signaller crossing large, low slow-moving vehicles animals automatic level crossings requiring local monitoring train drivers. (Score: 0.6864)
2.3.1.2 obstruction line level crossing occur when: (Score: 0.6860)
2.3.1.5 principle include pedestrian (footpath) crossings: interface members public trains level crossings referenced section 2.8 operational concept document. (Score: 0.6847)
2.3.1.3 measures reducing likelihood collisions level crossings developed railway industry cooperation relevant government bodies many years. responsibilities implementing maintaining measures defined legislation relatin